In [ ]:
import torch.nn as nn 
import torchvision.models as models 
import torch

# Load a pre-trained ResNet-18 model
custom_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# If you want to fine-tune the model for a different number of classes, you can modify the final layer
num_classes = 2
custom_model.fc = nn.Linear(custom_model.fc.in_features, num_classes)

# Example usage:
print(custom_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:

# Load a pre-trained ResNet-18 model
custom_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# If you want to fine-tune the model for a different number of classes, you can modify the final layer
num_classes = 2
custom_model.fc = nn.Linear(custom_model.fc.in_features, num_classes)

# Example usage:
print(custom_model)

In [25]:
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.data import random_split

# Define the image transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset
dataset = datasets.ImageFolder(root='garbage_classification', transform=transform)

# Split the dataset into training, validation, and testing sets
train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [22]:
for inputs, labels in train_loader:
    print(inputs.shape, labels)
    break

torch.Size([32, 3, 224, 224]) tensor([1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 0])


In [ ]:
from torch import optim

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(custom_model.parameters(), lr=0.01)

# Training loop with early stopping
num_epochs = 5
early_stopping_limit = 3
no_improvement_epochs = 0
best_loss = float('inf')

custom_model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = custom_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss}')
    
    # Check for early stopping
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        no_improvement_epochs = 0
    else:
        no_improvement_epochs += 1
    
    if no_improvement_epochs >= early_stopping_limit:
        print("Early stopping triggered")
        break

Epoch [1/5], Loss: 1.0123482581340906
Epoch [2/5], Loss: 0.9517830384500099
Epoch [3/5], Loss: 0.5325183796160149
Epoch [4/5], Loss: 0.42650364249041584
Epoch [5/5], Loss: 0.4501337670918667


In [ ]:
# Evaluation loop
custom_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = custom_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy: {100 * correct / total}%')

Accuracy: 81.87134502923976%


In [ ]:
torch.save(custom_model.state_dict(), 'trained_resnet18.keras')

In [7]:
import torch
import torch.nn as nn 
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import optim


In [9]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(CustomCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(64 * 28 * 28, num_classes)  # Assuming input images are 224x224

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Example usage:
custom_model = CustomCNN(num_classes=3)
print(custom_model)

CustomCNN(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=50176, out_features=3, bias=True)
)


In [3]:
# Define the image transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset
dataset = datasets.ImageFolder(root='garbage_classification', transform=transform)

# Split the dataset into training, validation, and testing sets
train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [4]:
for inputs, labels in train_loader:
    print(inputs.shape, labels)
    break

torch.Size([32, 3, 224, 224]) tensor([1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])


In [5]:
print(dataset.class_to_idx)

{'ewaste': 0, 'nonorganic': 1, 'organic': 2}


In [10]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(custom_model.parameters(), lr=0.01)

# Training loop with early stopping
num_epochs = 5
early_stopping_limit = 3
no_improvement_epochs = 0
best_loss = float('inf')

custom_model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = custom_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss}')
    
    # Check for early stopping
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        no_improvement_epochs = 0
    else:
        no_improvement_epochs += 1
    
    if no_improvement_epochs >= early_stopping_limit:
        print("Early stopping triggered")
        break

Epoch [1/5], Loss: 4.296997513784015
Epoch [2/5], Loss: 0.505765030781428
Epoch [3/5], Loss: 0.4858469340572323
Epoch [4/5], Loss: 0.45261464288215275
Epoch [5/5], Loss: 0.45342140785162


In [11]:
# Evaluation loop
custom_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = custom_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy: {100 * correct / total}%')

Accuracy: 81.6695059625213%


In [13]:
torch.save(custom_model.state_dict(), 'custom_cnn.h5')